In [11]:
# Visualization Bcl6-dependent genes in the ILC TRNs, using Waggoner Bcl6 KO gene expression heatmaps (log2(F.C.))
# _multiNet --> enables coordinated comparison Bcl6 networks per ILC subtype 
# Miraldi*, Pokrovskii* et al. (2018) "Characterization of transcriptional regulatory networks that drive the identities
# and functions of intestinal innate lymphoid cells"

# TO START: In the menu above, choose "Cell" --> "Run All", and network + heatmap will load
# NOTE: Default limits networks to TF-TF edges in top 1 TF / gene model (.93 quantile), to see the full 
# network hit "restore" (in the drop-down menu in cell below) and set threshold to 0 and hit "threshold"
# You can search for gene names in the search box below the network (hit "Match"), and find regulators ("targeted by")
# Change "canvas" to "SVG" (drop-down menu in cell below) to enable drag interactions with nodes & labels
# Change "SVG" to "canvas" to speed up layout operations
# More info about jp_gene_viz and user interface instructions are available on Github: 
# https://github.com/simonsfoundation/jp_gene_viz/blob/master/doc/dNetwork%20widget%20overview.ipynb

# Edge key: 
# Solid lines -- supported by ATAC-seq data and gene expression modeling
# Dotted lines -- supported by gene expression modeling only

In [12]:
# Choose an FDR cutoff to define Bcl6-dependent genes based on differential expression between KO and control
# Options are limited to FDR: "10" or "25" 
fdrCut = "25"

# directory containing gene expression data and network folder
directory = "."
# folder containing networks
netPath = "Bcl6KO_TRNs"

# networkInits is a list of tuples (network file, sample to color expression, title of network)
networkInits = [    
    ('Bcl6_KO_CCR6neg_ILC3_indirTargs_fdr' + fdrCut + '_sp.tsv','CCR6-ILC3_Bcl6_FC','CCR6-ILC3 Bcl6 KO TRN'),
    ('Bcl6_KO_ILC1_indirTargs_fdr' + fdrCut + '_sp.tsv','ILC1_Bcl6_FC','ILC1 Bcl6 KO TRN'),
    ('Bcl6_KO_NK_indirTargs_fdr' + fdrCut + '_sp.tsv','NK_Bcl6_FC','NK Bcl6 KO TRN'),       
    ('bcl6_unionNets_indirTargs_fdr' + fdrCut + '_sp.tsv','NK_Bcl6_FC','Union of ILC-subtype Bcl6 KO TRNs')]

# name of gene expression file
expressionFile = "GeneExpression/Bcl6_log2fcs_final.txt" 

# For ease of visualization, limit target genes to cytokines, chemokines, receptors. Hit "Restore" button to see
# all target genes in the TRN.
# geneListFile = "GeneSets/oxidativeDamage_Mapp.txt"
# geneListFile = "GeneSets/IL12_23_PathComm.txt"
# geneListFile = "GeneSets/IL2_PathCommMapp.txt"
# geneListFile = "GeneSets/MAPK_KeggGoMapp.txt"
# geneListFile = "GeneSets/chemokineReceptorsKlrs.txt"
geneListFile = ""

# Labe TF and gene nodes in the network?
labelsOn=True

# set model size, mean # of TFs / genes, with maximum being ~10 TFs / gene
tfsPerGene = 5

In [13]:
import sys 
import os.path
if ".." not in sys.path:
    sys.path.append("..")
from jp_gene_viz import dNetwork
dNetwork.load_javascript_support()
from jp_gene_viz import multiple_network
from jp_gene_viz import LExpression
LExpression.load_javascript_support()

In [14]:
networkList = list()  # this list contains heatmap-linked network objects

for networkInit in networkInits:
    networkFile = networkInit[0]
    curr = LExpression.LinkedExpressionNetwork()   
    fileName = directory + '/' + netPath + '/' + networkFile
    if os.path.isfile(fileName):
        curr.load_network(fileName)    
        networkList.append(curr) 
    else:
        print fileName + ' does not exist.'
    

('Reading network', './Bcl6KO_TRNs/Bcl6_KO_CCR6neg_ILC3_indirTargs_fdr25_sp.tsv')
('Loading saved layout', './Bcl6KO_TRNs/Bcl6_KO_CCR6neg_ILC3_indirTargs_fdr25_sp.tsv.layout.json')
('Reading network', './Bcl6KO_TRNs/Bcl6_KO_ILC1_indirTargs_fdr25_sp.tsv')
('Loading saved layout', './Bcl6KO_TRNs/Bcl6_KO_ILC1_indirTargs_fdr25_sp.tsv.layout.json')
('Reading network', './Bcl6KO_TRNs/Bcl6_KO_NK_indirTargs_fdr25_sp.tsv')
('Loading saved layout', './Bcl6KO_TRNs/Bcl6_KO_NK_indirTargs_fdr25_sp.tsv.layout.json')
('Reading network', './Bcl6KO_TRNs/bcl6_unionNets_indirTargs_fdr25_sp.tsv')
('Loading saved layout', './Bcl6KO_TRNs/bcl6_unionNets_indirTargs_fdr25_sp.tsv.layout.json')
Omitting edges, using canvas, and fast layout default because the network is large


In [15]:
# visualize the networks -- HARD CODED for 6 networks:
M = multiple_network.MultipleNetworks(
    [[networkList[0], networkList[1]],
    [networkList[2], networkList[3]]])
M.svg_width = 500
M.show()  

A Jupyter Widget

In [16]:
# Set network preferences
count = 0
for curr in networkList:
    networkInit = networkInits[count]
    # get title information + curr column for shading of figures
    currCol = networkInit[1]
    titleBase = networkInit[2]
    titleInf = titleBase 
    
    # set threshold
    threshhold = 1-float(tfsPerGene)/10 # because we're dealing with networks that have mean 10 TFs / gene        
    curr.network.threshhold_slider.value = threshhold
    curr.network.apply_click(None)
        
    # set title
    curr.network.title_html.value = titleInf

    # add labels
    curr.network.labels_button.value=labelsOn
    curr.network.draw_click(None)
    # Limit genes to target genes of interest and their putative regulators
    if geneListFile:
        geneIn = open(directory + '/' + geneListFile,'r')    
        geneList = list()
        for gene in geneIn:
            geneList.append(gene.strip('\n'))
        geneIn.close()

        curr.network.pattern_text.value = " ".join(geneList)
        curr.network.match_click(None)
        curr.network.targeted_click(None)

    curr.network.connected_only_click(None)
    curr.network.layout_dropdown.value = 'fruchterman_reingold'
    curr.network.layout_click()
    
    # Load heatmap
    curr.load_heatmap(directory + '/' + expressionFile)
    # color nodes according to a sample column in the gene expression matrix
    curr.gene_click(None)       
    curr.expression.col = currCol    
    curr.condition_click(None)
    
    count += 1